### Data load

In [1]:
base_dir = r'C:\Users\Yang\Desktop\project'

In [12]:
# csv파일(최종 데이터) 불러옴
import csv
import re

raw = []
f = open(base_dir + r'\data\final_data.csv', 'r', encoding='utf-8')
rdr = csv.reader(f)
for line in rdr:
    raw.append(line)
f.close()  

raw_text = []
label = []

# 핵심문장만 불러옴. label은 비관련 100이면 피해없다고 간주하고 나머진 피해라고 하였음.
for i in range(2, len(raw)):
    for j in range(30, 37):
        if(raw[i][j] is not ''):
            raw_text.append(raw[i][j])
            if(int(raw[i][29]) < 100 ):
                label.append('1')
            else:
                label.append('0')

In [7]:
# csv파일(직접 분류한거) 불러옴
import csv
import re
raw = []
f = open(base_dir+'\data\damage1100.csv', 'r', encoding='utf-8')
rdr = csv.reader(f)
for line in rdr:
    raw.append(line)
f.close()

raw_text.extend([i[0] for i in raw])
label.extend([i[1] for i in raw])

In [85]:
# txt파일(피해 비관련 데이터) 불러옴
import re
import os
import random
import numpy as np

directory = r'C:\Users\0000\Desktop\join\비관련'
files = os.listdir(directory) # 모든 파일 name
raw = [] # 모든 txt파일 내에 문장 다 가져옴

for filename in files:
    if ".txt" not in filename:
        continue
    file = open(directory+filename, 'r', encoding='utf-8')
    for line in file:
        raw.append(line)
    file.close()
    
raw_upper50 = [i for i in raw if len(i)>=50] # 50글자 이상
split_raw_upper50 = [] # 여러 문장을 split하여 단일 문장으로 나타냄.

for text in raw_upper50:
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    for i in sentences:
        p = re.split('[\.]',i)
        for j in p:
            split_raw_upper50.append(j)

count = 3000 # 랜덤하게 몇개 추출할건지
random.seed(100)
all_not_damage = [i for i in split_raw_upper50 if len(i)>=50] # 단일 문장들 중 50글자 이상만
random_not_damage = [random.choice(all_not_damage) for i in range(count)] # count만큼 랜덤하게 뽑음
zeros = np.zeros(count, dtype=int)
list_zeros = zeros.tolist()
label_not_damage = [str(i) for i in list_zeros] # 모두 0으로 라벨 설정
raw_text.extend(random_not_damage)
label.extend(label_not_damage)

In [68]:
import csv

csvfile = open(base_dir+'\data\data.csv', 'w', newline="", encoding='utf_8_sig')
csvwriter = csv.writer(csvfile)
for i in range(len(raw_text)):
    csvwriter.writerow([raw_text[i], label[i]])
csvfile.close()

In [15]:
# 빈 text 제거 & 특수문자 제거
text = []
for i in range(len(raw_text)):
    if(raw_text[i] is not ' '):
        newtext = re.sub('[-=+,#/\?:^$@*\"~&%!<>()"]', '', raw_text[i])
        text.append(newtext)

### 전체 7만개 문장 및 라벨 데이터

In [2]:
# csv파일(최종 데이터) 불러옴
import csv
import re

raw = []
f = open(base_dir + r'\data\data_70000.csv', 'r', encoding='utf-8')
rdr = csv.reader(f)
for line in rdr:
    raw.append(line)
f.close()  

raw_text = []
label = []

In [3]:
for i in raw:
    raw_text.append(i[0])
    label.append(i[1])

In [4]:
# 빈 text 제거 & 특수문자 제거
text = []
for i in range(len(raw_text)):
    if(raw_text[i] is not ' '):
        newtext = re.sub('[-=+,#/\?:^$@*\"~&%!<>()"]', '', raw_text[i])
        text.append(newtext)

### 품사 태깅

In [5]:
# 품사 태깅(주어진 텍스트를 형태소 단위로 나누고 나눠진 형태소를 해당하는 품사와 함께 리스트화)
# stemming(단어들을 원형으로 포현), normalization(표현 방법이 다른 단어들을 통합시켜 같은 단어로 만듬)
import json
import os 
from konlpy.tag import Okt

okt = Okt()

def tokenize(doc): # 형태소 분석기인 Okt를 이용해서 품사를 태깅시킴
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

os.chdir(base_dir) # 경로 설정

In [6]:
if os.path.isfile(base_dir+r'\word_model_70000\text_docs.json'): # 경로에 text_docs.json 파일이 존재하면 json파일을 만들지 않고 load함
    with open(base_dir+r'\word_model_70000\text_docs.json', encoding='utf-8') as f:
        text_docs = json.load(f)
else: # json파일이 없을 경우 text를 품사 태깅시킨 데이터로 변환시켜 json파일로 저장
    text_docs = [(tokenize(row)) for row in text]
    # JSON 파일로 저장
    with open(base_dir+r'\word_model_70000\text_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(text_docs, make_file, ensure_ascii=False, indent="\t")

In [35]:
# 태깅된 각 형태소들을 하나의 list로 합침
tokens = [t for d in text_docs for t in d]
print(len(tokens))

2141642


In [36]:
import nltk
from pprint import pprint

nltk_text = nltk.Text(tokens, name='NMSC')

# 전체 토큰의 개수
total_tokens = nltk_text.tokens
print(len(total_tokens))

# 중복을 제외한 토큰의 개수
set_total_tokens = list(set(nltk_text.tokens))
print(len(set_total_tokens))            

# 출현 빈도가 높은 상위 토큰 10개
#pprint(nltk_text.vocab().most_common(50))

2141642
41158


In [37]:
# 빈도수가 10개 이상인 token 확인.
most_common_tokens = nltk_text.vocab().most_common(len(set_total_tokens))
for i in range(len(set_total_tokens)):
    if(most_common_tokens[i][1]<10):
        index = i
        break
most_common_tokens_10 = most_common_tokens[:index]
len(most_common_tokens_10)

10578

## Embedding

In [8]:
size = 300

### Fasttext

In [11]:
from gensim.models import FastText
embedding_model = FastText(text_docs, size=size, window = 2, min_count=10, workers=4, iter=100, sg=1)

In [12]:
embedding_model.save(base_dir+'\word_model_70000\Fasttext_'+str(size)+'dim.model')

### Word2vec

In [9]:
# Word embedding(Word2vec 사용)
from gensim.models import Word2Vec
embedding_model = Word2Vec(text_docs, size=size, window = 2, min_count=10, workers=4, iter=100, sg=1)

C:\Users\Yang\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [10]:
embedding_model.save(base_dir+r'\word_model_70000\Word2vec_'+str(size)+'dim.model')

C:\Users\Yang\Anaconda3\envs\tensorflow\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


### Doc2vec

In [13]:
import gensim
import gensim.models.keyedvectors as word2vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

max_epochs = 100
vec_size = size
alpha = 0.025
tagged_title = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(text_docs)]
model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=10,
                dm =1)
model.build_vocab(tagged_title)

for epoch in range(max_epochs):
    model.train(tagged_title,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

C:\Users\Yang\Anaconda3\envs\tensorflow\lib\site-packages\gensim\models\doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
C:\Users\Yang\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


In [14]:
model.save(base_dir+r'\word_model_70000\Doc2vec_'+str(size)+'dim.model')

C:\Users\Yang\Anaconda3\envs\tensorflow\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


## 기사를 문장별로 토큰화

In [130]:
from konlpy.tag import Kkma
kkma = Kkma()

In [1]:
_tokens = [] # 기사를 문sentence장별로 토큰화한 결과값을 list에 저장
articles = list(set(text)) # 중복되는 기사 제거
for article in articles:
    sentence_tokens.append(kkma.sentences(article))    

NameError: name 'text' is not defined

## '피해'라는 단어와 top7 단어가 함께 들어있는 문장 추출

In [133]:
sentences_damage = []
for sentences in sentence_tokens:
    for sentence in sentences:
        isExist = False        
        for word_damage in words_damage:
            if isExist == False:
                if (word_damage in sentence) and ('피해' in sentence):
                    sentences_damage.append(sentence)
                    isExist = True

In [139]:
sentences_damage[10:20]

['또 일부 저지대 주택과 농경지가 침수되고 행인이 고립되는 등 10 여 건의 피해가 났습니다.',
 '온 열 질환자와 사망자가 급증하고 전력 소비량이 최대치를 경신하며 농작물과 가축 제조업 등에서 막대한 피해가 발생한다는 겁니다.',
 '다행히 사고 당시 다리를 지나던 차량이 많지 않아 피해는 발생하지 않았습니다.',
 '직원들이 재빨리 대피해 다친 사람은 없었지만 부화장 한 개 동이 모두 타 8천만 원의 재산피해가 났습니다.',
 '대전 통영 고속도로 하행선 경남 산청 IC 인근에서 그레이스 승합차에 불이 나 전소됐지만 인명 피해는 없었습니다.',
 '■ 고물 상 화재 …2 천만 원 피해 경남 김 해의 한 고물상에서도 불이 나 고물상 내 조립식 건물과 야적장 일부를 태워 소방서 추산 2천만 원 상당의 재산피해를 내고 20여 분 만에 꺼졌습니다.',
 '화재 당시 현장에 사람이 없어 인명피해는 없었습니다.',
 '[ 전 경 윤 침수 피해 횟집 주인 더위를 식히나 했지 저녁에는 그랬는데 난데없이 너무 많이 쏟아지니까 피해가 이만저만이 아니죠.]',
 '주택과 상가 침수 등 접수된 비 피해 신고는 강릉 지역에서만 80건이 넘습니다.',
 '이와 함께 불볕 더위로 전력 사용량이 크게 늘면서 축사 화재와 정전사고가 우려되므로 미리 점검해야 피해를 막을 수 있습니다.']

In [137]:
len(sentences_damage)

369

In [140]:
f = open(base_dir+r'\data\sentences_damage.csv', 'w', encoding='utf-8', newline='')
wr = csv.writer(f)
for i in sentences_damage:
    wr.writerow([i])
f.close()